In [1]:
from flask import Flask, jsonify

app = Flask(__name__)

@app.route("/")
def hello_world():
    return  'Hello, World!'

@app.route("/test")
def hello_json():
    data = {'name':'alex', 'age' : 30}
    return jsonify(data)

if __name__ == "__main__":
    app.run(debug = True, host = '0.0.0.0')

SystemExit: 1

To exit: use 'exit', 'quit', or Ctrl-D.


In [7]:
import requests
import json

res = requests.get("http://ec2-54-213-224-243.us-west-2.compute.amazonaws.com:5000/test")
content =json.loads(res.content)
print content

{u'age': 30, u'name': u'alex'}


In [3]:
from pymongo import MongoClient
from exer_connection_db import mongo_account

mongo = MongoClient('ec2-54-213-224-243.us-west-2.compute.amazonaws.com', 27017)

mongo.the_database.authenticate(mongo_account['id'], mongo_account['pw'], source='test')
person = mongo.test.person

result = person.find_one({'name': 'alex'})
print result

{u'gender': u'M', u'age': 30.0, u'_id': ObjectId('582ef58bd55df5dbfd6328fa'), u'name': u'alex', u'regdate': datetime.datetime(2016, 11, 18, 12, 35, 23, 360000)}


In [22]:
import requests
from bs4 import BeautifulSoup
import re
from pymongo import MongoClient
from exer_connection_db import mongo_account

mongo = MongoClient('ec2-54-213-224-243.us-west-2.compute.amazonaws.com' ,27017)
mongo.the_database.authenticate(mongo_account['id'], mongo_account['pw'], source = 'test')
actor = mongo.test.actor

Data1 = {}
Data1['section'] = 'actor'
Data1['period_start'] = '2016-10'
Data1['gender'] = 'all'

for i in range(1,6):
    Data1['page'] = i
    response = requests.post("http://www.cine21.com/rank/person/content", data = Data1)
    soup = BeautifulSoup(response.content)

    actor_names =  soup.find_all("div", attrs = {"class" : "name"})
    points = soup.find_all('strong')
    for name, point in zip(actor_names, points):
        sub_name = name.get_text()
        actor_name = re.sub("\(.+\)", "", sub_name)
        str_point = point.get_text()
        int_point = int(re.sub(",", "", point.get_text()))
        
        actor.insert_one({'name' : actor_name, 'score' : int_point})
        
    

In [23]:
import requests
from bs4 import BeautifulSoup
import re
from pymongo import MongoClient
from exer_connection_db import mongo_account

mongo = MongoClient('ec2-54-213-224-243.us-west-2.compute.amazonaws.com' ,27017)
mongo.the_database.authenticate(mongo_account['id'], mongo_account['pw'], source = 'test')
actor = mongo.test.actor2

Data1 = {}
Data1['section'] = 'actor'
Data1['period_start'] = '2016-10'
Data1['gender'] = 'all'

actor_point = []
for i in range(1,6):
    Data1['page'] = i
    response = requests.post("http://www.cine21.com/rank/person/content", data = Data1)
    soup = BeautifulSoup(response.content)

    actor_names =  soup.find_all("div", attrs = {"class" : "name"})
    points = soup.find_all('strong')
    for name, point in zip(actor_names, points):
        sub_name = name.get_text()
        actor_name = re.sub("\(.+\)", "", sub_name)
        str_point = point.get_text()
        int_point = int(re.sub(",", "", point.get_text()))
        
        actor_point.append({'name' : actor_name, 'score' : int_point})

actor.insert_many(actor_point)
    

In [24]:
mongo = MongoClient('ec2-54-213-224-243.us-west-2.compute.amazonaws.com' ,27017)
mongo.the_database.authenticate(mongo_account['id'], mongo_account['pw'], source = 'test')
actor = mongo.test.actor

result = actor.find_one({"name" : "유해진"})
print result

{u'_id': ObjectId('58326d0c65acca0238349c68'), u'score': 124523, u'name': u'\uc720\ud574\uc9c4'}


In [30]:
mongo = MongoClient('ec2-54-213-224-243.us-west-2.compute.amazonaws.com' ,27017)
mongo.the_database.authenticate(mongo_account['id'], mongo_account['pw'], source = 'test')
actor = mongo.test.actor

actor.update_one({'name' : '유해진'}, {'$set' : {'score' : 124524}})

actor.update_many({'name' : '유해진'}, {'$set' : {'score' : 124524}})
result = actor.find_one({'name' : '유해진'})
print result

{u'_id': ObjectId('58326d0c65acca0238349c68'), u'score': 124524, u'name': u'\uc720\ud574\uc9c4'}


In [31]:
mongo = MongoClient('ec2-54-213-224-243.us-west-2.compute.amazonaws.com' ,27017)
mongo.the_database.authenticate(mongo_account['id'], mongo_account['pw'], source = 'test')
actor = mongo.test.actor

result = actor.find().sort('name', -1)
for doc in result:
    print doc['name']

황정민
한지민
최승호
주지훈
조한철
조윤희
조복래
정우성
정성화
정만식
전혜빈
임지연
이효제
이준
이정현
이용녀
이수빈
이동휘
이다윗
윤지혜
유해진
유지태
엄태구
신은수
송강호
류덕환
김희원
김해곤
김원해
김민상
김기춘
권해효
곽도원
공유
강동원


In [32]:
result = actor.delete_one({'name' : '황정민'})
print result.deleted_count

1


In [33]:
result = actor.find().sort('name', -1)
for doc in result:
    print doc['name']

한지민
최승호
주지훈
조한철
조윤희
조복래
정우성
정성화
정만식
전혜빈
임지연
이효제
이준
이정현
이용녀
이수빈
이동휘
이다윗
윤지혜
유해진
유지태
엄태구
신은수
송강호
류덕환
김희원
김해곤
김원해
김민상
김기춘
권해효
곽도원
공유
강동원
